<a href="https://colab.research.google.com/github/jiw000/yes24_book/blob/main/yes24_%EA%B2%BD%EC%A0%9C%2C%EA%B2%BD%EC%98%81_%EC%B1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 유사도 기반 책 추천, 단어 군집화, 책의 category 예측하기
1. YES24의 경제,경영 책 데이터 수집
2. td_idf를 사용하여 원하는 책과 유사한 내용의 책을 소개해주는 모델을 만들기
3. K_means를 활용하여 책 소개 단어별 군집화
4. randomforest를 활용한 category 예측


### 데이터 수집

In [ ]:
!pip3 install beautifulsoup4 -U
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 128 kB 34.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
answer = []


categories = [
("001001025007", "경제"),
("001001025008","경영"),
("001001025010","투자/재태크"),
("001001025009","마케팅/세일즈"),
("001001025001","CEO/비즈니스맨"),
("001001025011","인터넷비즈니스"),
("001001025006","총량/연감")

]
max_page = 50

for cate_code, cate_name in categories: 

    for i in tqdm(range(1, max_page+1)):
        url = f"http://www.yes24.com/24/Category/Display/{cate_code}?FetchSize=40&PageNumber={i}"
        html = requests.get(url).text
        soup = BeautifulSoup(html, "html.parser")

        string = "#category_layout > ul > li > div > div.goods_info > div.goods_name > a:nth-child(2)"
        temp = soup.select(string)
        for line in temp:
            res_detail = {}
            res_detail['name']=line.text
            res_detail['link']= "http://www.yes24.com" + line['href']
            res_detail["category"] = cate_name
            answer.append(res_detail)


In [ ]:
answer[:10]

[{'name': '트렌드 코리아 2023 ',
  'link': 'http://www.yes24.com/Product/Goods/113416767',
  'category': '경제'},
 {'name': '웹 3.0 넥스트 이코노미',
  'link': 'http://www.yes24.com/Product/Goods/115409192',
  'category': '경제'},
 {'name': '머니 트렌드 2023',
  'link': 'http://www.yes24.com/Product/Goods/115258401',
  'category': '경제'},
 {'name': '창발의 시대',
  'link': 'http://www.yes24.com/Product/Goods/115097094',
  'category': '경제'},
 {'name': '레버리지',
  'link': 'http://www.yes24.com/Product/Goods/74208134',
  'category': '경제'},
 {'name': '부의 추월차선 (10주년 스페셜 에디션)',
  'link': 'http://www.yes24.com/Product/Goods/106369008',
  'category': '경제'},
 {'name': '이코노미스트 2023 세계대전망 ',
  'link': 'http://www.yes24.com/Product/Goods/115550520',
  'category': '경제'},
 {'name': '그레이트 리세션 2023년 경제전망',
  'link': 'http://www.yes24.com/Product/Goods/114854198',
  'category': '경제'},
 {'name': '넛지: 파이널 에디션 ',
  'link': 'http://www.yes24.com/Product/Goods/110157236',
  'category': '경제'},
 {'name': '미스터 마켓 2023',
  'link': 'http://ww

In [ ]:

for single in tqdm(answer):
    link = single["link"]

    url = link
    
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")

    string = "#infoset_introduce > div.infoSetCont_wrap > div.infoWrap_txt > div" 
    temp = soup.select(string)

    if temp:
        single['text'] = temp[0].text.strip()
    else:
        single["text"] = None
                                     
# break

100%|██████████| 13244/13244 [3:24:55<00:00,  1.08it/s]


In [ ]:
df = pd.DataFrame(answer)
df.to_csv("book_list")
df

,name,link,category,text
0,트렌드 코리아 2023,http://www.yes24.com/Product/Goods/113416767,경제,RABBIT JUMP 더 높은 도약을 준비하는 검은 토끼의 해 세계화의 종말...
1,웹 3.0 넥스트 이코노미,http://www.yes24.com/Product/Goods/115409192,경제,"돈은 사라지지 않는다우리가 모르는 곳으로 이동하고 있을 뿐!* 『김미경의 리부트』,..."
2,머니 트렌드 2023,http://www.yes24.com/Product/Goods/115258401,경제,None
3,창발의 시대,http://www.yes24.com/Product/Goods/115097094,경제,"이 책은 현대 세계가 어떻게 형성되었는지 그 흥미로운 역사로 안내하는경제적, 문화적..."
4,레버리지,http://www.yes24.com/Product/Goods/74208134,경제,None
...,...,...,...,...
13239,한국의 장기통계 2,http://www.yes24.com/Product/Goods/58259603,총량/연감,None
13240,한국의 장기통계 1,http://www.yes24.com/Product/Goods/58259595,총량/연감,"일반인은 물론, 연구자들도 자신의 전공을 넘어 다른 분야에서 원하는 통계를 찾기가 ..."
13241,2016 애니메이션 산업백서,http://www.yes24.com/Product/Goods/36071691,총량/연감,2015년부터 2016년 상반기까지 애니메이션 산업의 주요 동향 및 이슈들을 소개했...
13242,국내 자산유동화증권의 스프레드 결정요인 분석,http://www.yes24.com/Product/Goods/35907886,총량/연감,None


In [ ]:
df[:50]

,name,link,category,text
0,트렌드 코리아 2023,http://www.yes24.com/Product/Goods/113416767,경제,RABBIT JUMP 더 높은 도약을 준비하는 검은 토끼의 해 세계화의 종말...
1,웹 3.0 넥스트 이코노미,http://www.yes24.com/Product/Goods/115409192,경제,"돈은 사라지지 않는다우리가 모르는 곳으로 이동하고 있을 뿐!* 『김미경의 리부트』,..."
2,머니 트렌드 2023,http://www.yes24.com/Product/Goods/115258401,경제,None
3,창발의 시대,http://www.yes24.com/Product/Goods/115097094,경제,"이 책은 현대 세계가 어떻게 형성되었는지 그 흥미로운 역사로 안내하는경제적, 문화적..."
4,레버리지,http://www.yes24.com/Product/Goods/74208134,경제,None
5,부의 추월차선 (10주년 스페셜 에디션),http://www.yes24.com/Product/Goods/106369008,경제,미국 아마존 금융ㆍ사업 분야 1위 국내 유명서점 10년간 종합 베스트셀러가장 빠르게...
6,이코노미스트 2023 세계대전망,http://www.yes24.com/Product/Goods/115550520,경제,"이코노미스트 『2023 세계대전망』 한국어판 독점 출간전 세계 90개국 이상, 25..."
7,그레이트 리세션 2023년 경제전망,http://www.yes24.com/Product/Goods/114854198,경제,"“리세션의 시대,최악의 경기침체에도 기회는 있다”2023년, 쉽게 끝나지 않을 경기..."
8,넛지: 파이널 에디션,http://www.yes24.com/Product/Goods/110157236,경제,"기후변화 · 코로나19 극복에서 슬러지 · 연금플랜 설계까지,21세기를 송두리째 바..."
9,미스터 마켓 2023,http://www.yes24.com/Product/Goods/115781720,경제,"“2023년, 부의 재분배가 일어난다!”침체의 공포에 숨겨진 투자의 본질과 신호초강..."


### 데이터 불러오기

In [ ]:
df = pd.read_csv('book_list',index_col = 0)
df

,name,link,category,text
0,트렌드 코리아 2023,http://www.yes24.com/Product/Goods/113416767,경제,RABBIT JUMP 더 높은 도약을 준비하는 검은 토끼의 해 세계화의 종말...
1,웹 3.0 넥스트 이코노미,http://www.yes24.com/Product/Goods/115409192,경제,"돈은 사라지지 않는다우리가 모르는 곳으로 이동하고 있을 뿐!* 『김미경의 리부트』,..."
2,머니 트렌드 2023,http://www.yes24.com/Product/Goods/115258401,경제,NaN
3,창발의 시대,http://www.yes24.com/Product/Goods/115097094,경제,"이 책은 현대 세계가 어떻게 형성되었는지 그 흥미로운 역사로 안내하는경제적, 문화적..."
4,레버리지,http://www.yes24.com/Product/Goods/74208134,경제,NaN
...,...,...,...,...
11370,키워드 광고 이기는 전략,http://www.yes24.com/Product/Goods/6959136,인터넷비즈니스,개인 쇼핑몰에서 대기업까지 키워드 광고 때문에 머리가 아픈 마케터를 위한 실전 전략...
11371,유저,http://www.yes24.com/Product/Goods/6771372,인터넷비즈니스,성공하고 싶다면 사용자의 마음을 빼앗아라! 디지털 마케팅 컨설팅 업체 휴즈(HUG...
11372,동대문시장 원도매 사입가이드,http://www.yes24.com/Product/Goods/6738869,인터넷비즈니스,동대문시장 베스트 원도매 180곳 완전 공개!2007년 출간 이후 초보 쇼핑몰 운영...
11373,나는 G마켓 옥션에서도 팔고 이베이에서도 판다,http://www.yes24.com/Product/Goods/6721548,인터넷비즈니스,이베이 전문 커뮤니티 ‘이베이최강카페’를 통해서 지난 5년간 쌓아온 이베이 탑레이티...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11375 entries, 0 to 11374
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      11375 non-null  object
 1   link      11374 non-null  object
 2   category  11374 non-null  object
 3   text      9241 non-null   object
dtypes: object(4)
memory usage: 444.3+ KB


In [ ]:
# 도서명 기준 중복값 확인
df[df.duplicated(subset = 'name')]

,name,link,category,text
211,경제기사 궁금증 300문 300답,http://www.yes24.com/Product/Goods/84936246,경제,NaN
236,"비트코인, 지혜의 족보",http://www.yes24.com/Product/Goods/90105791,경제,NaN
262,경제지표 정독법,http://www.yes24.com/Product/Goods/59729293,경제,"왜 항상 부자들은 남보다 한발 빠를까?경제지표에 숨겨진 돈과 경제, 부의 흐름을 읽..."
539,작은 것이 아름답다,http://www.yes24.com/Product/Goods/262509,경제,『작은 것이 아름답다』는 실천적 경제학자이자 환경운동가로 유명한 슈마허의 역작으로 ...
882,거대한 전환,http://www.yes24.com/Product/Goods/38005358,경제,"트럼프, 브렉시트. 4차 산업혁명, 미중간 힘겨루기 등 전 세계는 '거대한 전환'의..."
...,...,...,...,...
11342,친절한 이베이 창업 & 운영 가이드,http://www.yes24.com/Product/Goods/9403174,인터넷비즈니스,"『친절한 이베이 창업&운영 가이드』의 최신 개정판으로, 2013년 8월을 기점으로 ..."
11359,오픈마켓 광고 이기는 전략,http://www.yes24.com/Product/Goods/7195135,인터넷비즈니스,"광고, 피할 수 없다면 승자가 되라! 오픈마켓에서 상품의 판매량은 상품의 노출 순위..."
11370,키워드 광고 이기는 전략,http://www.yes24.com/Product/Goods/6959136,인터넷비즈니스,개인 쇼핑몰에서 대기업까지 키워드 광고 때문에 머리가 아픈 마케터를 위한 실전 전략...
11372,동대문시장 원도매 사입가이드,http://www.yes24.com/Product/Goods/6738869,인터넷비즈니스,동대문시장 베스트 원도매 180곳 완전 공개!2007년 출간 이후 초보 쇼핑몰 운영...


In [ ]:
# 도서명 기준 중복값 제거
df.drop_duplicates(subset = 'name', keep = 'first', inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9284 entries, 0 to 11374
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      9284 non-null   object
 1   link      9283 non-null   object
 2   category  9283 non-null   object
 3   text      7610 non-null   object
dtypes: object(4)
memory usage: 362.7+ KB


### 데이터 전처리

In [ ]:
! pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 31.6 MB/s 
     |████████████████████████████████| 465 kB 84.4 MB/s 


In [ ]:
# Okt 불러오기
import konlpy.tag
okt = konlpy.tag.Okt()

In [ ]:
# nan 제거
df = df.dropna()

In [ ]:
# 명사 단어 전처리
df['text'] = df['text'].map(lambda x: okt.nouns(x))

<ipython-input-10-c0cba3cedf0b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].map(lambda x: okt.nouns(x))


In [ ]:
# 리스트 > 문자열 변환
df['text'] = df['text'].map(lambda x: ' '.join(x))

<ipython-input-11-cb9b13356bc9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].map(lambda x: ' '.join(x))


In [ ]:
# TF-IDF
vectorizer = TfidfVectorizer(min_df=1, ngram_range=(1,1)) # 최소 빈도값, 단어의 묶음 범위 설정
tfidf = vectorizer.fit_transform(df['text'])

In [ ]:
# DTM 생성
dtm = np.array(tfidf.todense())

In [ ]:
# 이름 불러 오기
feature_names = vectorizer.get_feature_names()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
df1 = pd.DataFrame(data = dtm, columns = feature_names)
df1

,가가,가각,가감,가게,가격,가격결정,가격탄력성,가계,가계도,가계부,...,힐러리,힐링,힐스테이트,힘겨루기,힘껏,힘스,힘팀,힙스터,힙하,힙합
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df1.loc[df1["가격결정"] > 0, :]

,가가,가각,가감,가게,가격,가격결정,가격탄력성,가계,가계도,가계부,...,힐러리,힐링,힐스테이트,힘겨루기,힘껏,힘스,힘팀,힙스터,힙하,힙합
34,0.0,0.0,0.0,0.0,0.061101,0.595522,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
680,0.0,0.0,0.0,0.0,0.561826,0.091265,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
933,0.0,0.0,0.0,0.0,0.089406,0.174279,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4832,0.0,0.0,0.0,0.0,0.000000,0.109441,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4969,0.0,0.0,0.0,0.0,0.000000,0.282009,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7154,0.0,0.0,0.0,0.0,0.000000,0.416851,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
feature_names

['가가',
 '가각',
 '가감',
 '가게',
 '가격',
 '가격결정',
 '가격탄력성',
 '가계',
 '가계도',
 '가계부',
 '가계부채',
 '가계약',
 '가고메',
 '가고시마',
 '가공',
 '가공업',
 '가공품',
 '가교',
 '가구',
 '가권',
 '가급',
 '가기',
 '가까스로',
 '가까이',
 '가끔',
 '가나',
 '가나다순',
 '가난',
 '가난뱅이',
 '가내',
 '가넷',
 '가늠',
 '가능',
 '가능성',
 '가다가',
 '가닥',
 '가담',
 '가당',
 '가도',
 '가독성',
 '가동',
 '가두',
 '가두리',
 '가드',
 '가득',
 '가든',
 '가등기',
 '가디언',
 '가뜩이나',
 '가라',
 '가량',
 '가려진',
 '가령',
 '가로',
 '가로막',
 '가로수',
 '가로축',
 '가루받이',
 '가르시아',
 '가르침',
 '가름',
 '가망',
 '가맹',
 '가면',
 '가명',
 '가모',
 '가문',
 '가뭄',
 '가미',
 '가방',
 '가변',
 '가병',
 '가봉',
 '가부',
 '가부장',
 '가부장제',
 '가분',
 '가브리엘',
 '가비아',
 '가쁘',
 '가사',
 '가사노동',
 '가산',
 '가산금',
 '가상',
 '가상공간',
 '가상세계',
 '가상재화',
 '가상현실',
 '가상화',
 '가설',
 '가설검증',
 '가성',
 '가세',
 '가속',
 '가속도',
 '가솔린',
 '가수',
 '가스',
 '가슴',
 '가슴속',
 '가습기',
 '가시',
 '가식',
 '가신',
 '가십',
 '가안',
 '가압류',
 '가액',
 '가야',
 '가업',
 '가업승계',
 '가열',
 '가오',
 '가오루',
 '가완디',
 '가요',
 '가용',
 '가용성',
 '가우',
 '가우스전자',
 '가운',
 '가운데',
 '가위',
 '가위바위보',
 '가율',
 '가은',
 '가을',
 '가의',
 '가이던스',
 '가이드',
 '가이

In [ ]:
df1.iloc[1, :].sum()

7.453288551312419

In [ ]:
# 빈도 평균 확인
meanss = df1.loc[:,:].mean()
Means = pd.DataFrame(meanss)
Means = Means.sort_values(by=0) # 오름차순으로 정렬
print (Means.loc[Means[0] >= 0.000050])

            0
조우   0.000050
이해진  0.000050
하녀   0.000050
삐약   0.000050
리포터  0.000050
..        ...
사람   0.021684
경제   0.022734
마케팅  0.023328
기업   0.025757
투자   0.031384

[10230 rows x 1 columns]


In [ ]:
# 0.000010 이하인 값 제거
#23053 - 9986 = 13067
stopwords = Means.iloc[:13067,:]
stopwords = stopwords.index
stopwords = stopwords.values.tolist()
print(stopwords)

['각본가', '루카스필름', '라따뚜이', '복제판', '그래픽스', '잠자코', '아카데미상', '온상', '정사각형', '중추신경', '박스오피스', '학우', '트러블', '소집', '애니메이터', '게피', '워녹', '월트라', '라푼젤', '시나리오작가', '신흥기업', '동료평가', '컴퓨터그래픽', '장편', '문책', '상업성', '통구', '명패', '빈둥거리', '윈도', '조처', '영가로', '루카스', '데밍', '패란', '진시황', '유가사', '왕안석', '월나라', '추대', '양생', '고초', '원성', '상장군', '장사치', '변법', '진나라', '겨울왕국', '래스터', '중고나라', '품앗이', '혼선', '소굴', '혼잡', '밀화', '심장병', '행정도시', '슬럼가', '종교시설', '지하상가', '보건관리', '구의역', '스크린도어', '터미널', '공연장', '반신반의', '왜소', '자쪽', '은수저', '엑싯할', '피니시', '김용균', '하인리히', '법석', '들썩거릴', '외양간', '돌려막기', '한테라', '에버랜드', '발본색원', '코닝', '법학과', '압사', '법질서', '성신여대', '스톱', '효선', '뒷걸음', '등교', '트기', '김진선', '자금지원', '페인트', '리폼', '수출량', '정미', '지게차', '플러그', '겉핡기', '컨설팅하듯', '수나', '석비', '감격', '국무조정실장', '행정고시', '서관', '방만', '술회', '정현', '구본현', '순제', '강혜영', '박주성', '김도훈', '장재호', '아르테미스', '죽스', '필팩', '징둥', '재태크', '전신주', '구정', '풀꽃', '나태주', '서대문구', '가로수', '자치구', '급식', '광역의회', '도봉구', '결근', '인부', '근로장려금', '태양계', '점성술', '운석', '우주론', '속력', '창조론', '낙하', '우주왕복선', '우주망원경', '다누리

In [ ]:
# 불용어 제거 
df1 = df1.drop(stopwords, axis=1)
df1

,가가,가감,가게,가격,가격결정,가계,가계부,가계부채,가계약,가공,...,희토류,히든,히로,히스,히스토리,히트,히틀러,힌트,힐러리,힐링
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 코사인 유사도 기준 책 추천

In [ ]:
# 코사인 유사도 구하기
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(df1, df1)

In [ ]:
# 도서명 기준 인덱스 설정 인덱스
indexx = pd.Series(df.index, index=df['name']); indexx

name
트렌드 코리아 2023                         0
웹 3.0 넥스트 이코노미                       1
창발의 시대                               3
부의 추월차선 (10주년 스페셜 에디션)               5
이코노미스트 2023 세계대전망                    6
                                 ...  
포스트 코로나, 경제 사회 변화에 대한 전망과 시사점    13235
대한민국 100대 프랜차이즈 2019 연감          13237
한국의 장기통계 1                       13240
2016 애니메이션 산업백서                  13241
의약 바이오산업과 기술개발 동향                13243
Length: 8671, dtype: int64

In [ ]:
# 도서명 확인 
set(df['name'])

{'2023 전국 금은방·귀금속 주소록 CD',
 '역사에서 리더를 만나다',
 '물가의 경제학',
 '주식 투자 잘하는 사람들의 7가지 무기',
 '최적합 ICDL 디지털 마케팅',
 '공공기관에서 일하고 싶다구요?',
 '퍼스널 브랜딩 레볼루션',
 '내 은퇴통장 사용설명서',
 '지방 소멸',
 '나는 인생의 아주 기본적인 것부터 바꿔보기로 했다',
 '탁월한 조정자들',
 '담대한 전환 대한민국 산업미래전략 2030',
 'e-비즈니스 노트',
 '2022 주택과 세금',
 '특허실무지식: 특허 문서론 + 논증과 설득 (세트)',
 '메타버스와 돈의 미래',
 '잘나가는 MD',
 '한경트렌드 틈틈이 가족여행',
 '1천만 원으로 시작하는 빌라 투자 비법',
 '어서와~ HR은 처음이지?',
 '요즘 세대와 원 팀으로 일하는 법',
 '즐거운 농업의 시작, 스마트팜 이야기',
 '2011 물류산업총람',
 'Amway story 암웨이 스토리',
 'ESG 혁명이 온다',
 '전자상거래에서의 선점효과에 관한 새로운 접근과 분석',
 '1%의 차이가 부자를 만든다 ',
 '클릭 디지털 시대 직업 혁명',
 '더 클래식 벤저민 그레이엄',
 '스마트 콘텐츠 마케팅',
 '인간적인 브랜드가 살아남는다',
 '자본 3-1',
 '디지털 마케터로 일하고 있습니다',
 '취업규칙 · 근로계약의 작성과 운용',
 '업무에 날개를 달아주는 스마트워크 노하우',
 'IMF와 세계경제 이야기',
 '무엇이 행동하게 하는가',
 'AI 소사이어티 AI Society',
 '스티브 니슨의 캔들차트 투자기법',
 '성공을 위한 고객 관리 노트',
 '초기 스타트업을 위한 비즈니스 모델 30문 30답',
 '프랭클린 루스벨트의 온 아워 웨이',
 '시스템의 힘',
 '엔터테인먼트 통합 마케팅',
 '나는 ( *** ) 팀장이다',
 '줌 & 영상 편집',
 '전자책의 충격',
 '계정과목별 일반회계와 세무해설 2022',
 '현재 미래 트렌드 2022',
 '신화는 없다',
 

In [ ]:
# 마음에 드는 책 인덱스 추출
idx = indexx['물가의 경제학']
print(idx)

1345


In [ ]:
# 선택한 책과 다른 책의 유사도 확인
sim_scores = list(enumerate(cosine_sim[idx]))

In [ ]:
# 유사도가 높은 순으로 책 정렬
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

In [ ]:
# 유사도 높은 10개의 책
sim_scores = sim_scores[1:11]
sim_scores

[(1044, 0.16255558071911924),
 (2609, 0.1436130826376048),
 (7634, 0.14294326232482227),
 (236, 0.14277424179866682),
 (835, 0.1421110208913903),
 (6517, 0.13604197825992081),
 (7867, 0.13467464468585547),
 (7929, 0.1289886100934069)]

In [ ]:
# 가장 유사한 10개의 책의 인덱스
book_indices = [i[0] for i in sim_scores]

In [ ]:
# 유사한 책 제목 출력
df['name'].iloc[book_indices]

1323                글로벌라이징 캐피털
3311                        파워
11404                    메가챌린지
329      최배근 대한민국 대전환 100년의 조건
1074               베이징의 애덤 스미스
9629               국제회의 기획 운영 
11724       손에 잡히는 IT Business
11797              IP창업 성공 가이드
Name: name, dtype: object

In [ ]:
# 함수로 만들어보기
def recommendations(title, cosine_sim=cosine_sim):

    # 선택한 책의 인덱스를 가져오기
    idx = indexx[title]

    # 검색한 책과 모든 책의 유사도를 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도가 높은 순으로 책을 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 책을 가져오기
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 책의 인덱스를 가져오기
    book_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 책의 제목을 반환
    return df['도서명'].iloc[book_indices]

### K-means를 활용한 단어 군집화

In [ ]:
from sklearn.cluster import KMeans
import plotly.express as px

In [ ]:
# 최적 군집수 찾기

def elbow(X):
    sse = []

    for i in range(1,6):
        km = KMeans(n_clusters=i)
        km.fit(X)
        sse.append(km.inertia_)

    fig = px.line(sse)
    fig.show()

elbow(df1)

In [ ]:
# cluster 수 3개로 결정
km = KMeans(n_clusters=3).fit(df1)
km.cluster_centers_

array([[ 3.71426936e-04,  6.37969144e-04,  3.51477207e-03, ...,
         4.42009498e-04,  2.49296058e-04,  1.68502595e-04],
       [ 1.93379569e-03,  9.10203480e-04,  1.10729337e-04, ...,
         6.25244009e-04, -5.42101086e-20, -2.30392962e-19],
       [ 4.50099561e-04,  1.90008398e-04,  1.95825054e-04, ...,
         7.47766686e-04,  7.40081836e-05,  1.42301535e-18]])

In [ ]:
# 클러스터링을 통해 군집
predict = pd.DataFrame(km.predict(df1))
predict.columns = ['Cluster']
predict.head(3)

,Cluster
0,2
1,2
2,2


In [ ]:
# 예측 데이터 병합
result = pd.concat([df1, predict], axis=1).reset_index(drop=True)
result

,가가,가감,가게,가격,가격결정,가계,가계부,가계부채,가계약,가공,...,히든,히로,히스,히스토리,히트,히틀러,힌트,힐러리,힐링,Cluster
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
# 각 요소들 군집화
c_result = km.cluster_centers_
c_result = pd.DataFrame(data=c_result)
c_result.columns = df1.columns
c_result

,가가,가감,가게,가격,가격결정,가계,가계부,가계부채,가계약,가공,...,희토류,히든,히로,히스,히스토리,히트,히틀러,힌트,힐러리,힐링
0,0.000371,0.000638,0.003515,0.002814,2.297689e-04,0.000185,0.000577,-6.830474e-18,1.085058e-04,0.000568,...,-9.242824e-18,2.001211e-04,1.542336e-04,2.647703e-04,0.000022,7.804976e-04,7.742037e-05,0.000442,2.492961e-04,1.685026e-04
1,0.001934,0.000910,0.000111,0.010556,1.029992e-18,0.000308,0.000503,7.614436e-04,1.084202e-19,0.000271,...,5.692061e-19,6.505213e-19,9.486769e-20,1.870249e-18,0.000218,-2.168404e-19,5.827587e-19,0.000625,-5.421011e-20,-2.303930e-19
2,0.000450,0.000190,0.000196,0.004093,2.997732e-04,0.001400,0.000149,7.842517e-04,-1.734723e-18,0.000257,...,7.745554e-04,1.924459e-18,5.849467e-05,4.448925e-05,0.000102,3.166909e-04,4.895354e-05,0.000748,7.400818e-05,1.423015e-18


### Randomforest를 활용하여 category 예측하기

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
category = df[['category']]
category = category.reset_index()
category = category.drop('index', axis=1)
category

,category
0,경제
1,경제
2,경제
3,경제
4,경제
...,...
8666,총량/연감
8667,총량/연감
8668,총량/연감
8669,총량/연감


In [ ]:
total = pd.concat([df1, category],axis = 1)
total

,가가,가감,가게,가격,가격결정,가경제,가계,가계도,가계부,가계부채,...,히로,히스,히스토리,히트,히틀러,히피,힌트,힐러리,힐링,category
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,경제
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,경제
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,경제
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,경제
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,경제
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094662,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,총량/연감
8667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,총량/연감
8668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,총량/연감
8669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,총량/연감


In [ ]:
train = total.iloc[:6000]
train

,가가,가감,가게,가격,가격결정,가경제,가계,가계도,가계부,가계부채,...,히로,히스,히스토리,히트,히틀러,히피,힌트,힐러리,힐링,category
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,경제
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,경제
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,경제
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,경제
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,경제
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CEO/비즈니스맨
5996,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CEO/비즈니스맨
5997,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CEO/비즈니스맨
5998,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CEO/비즈니스맨


In [ ]:
test = total.iloc[6000:8671]
test

,가가,가감,가게,가격,가격결정,가경제,가계,가계도,가계부,가계부채,...,히로,히스,히스토리,히트,히틀러,히피,힌트,힐러리,힐링,category
6000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CEO/비즈니스맨
6001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CEO/비즈니스맨
6002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CEO/비즈니스맨
6003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CEO/비즈니스맨
6004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CEO/비즈니스맨
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094662,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,총량/연감
8667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,총량/연감
8668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,총량/연감
8669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,총량/연감


In [ ]:
train_x=train.drop('category', axis=1)
test = test.drop('category', axis=1)
train_y=train[['category']]
test_x=test

In [ ]:
# 모델링 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, 
                                                    stratify=train_y, test_size=0.25,
                                                    random_state = 40)

clf=RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred=clf.predict(X_val)

print(f"accuracy: {accuracy_score(y_val['category'], y_pred)}")


<ipython-input-35-89ec0a82b73a>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


accuracy: 0.724
